# IPython magic extension version_information

Use the '%version_information' IPython magic extension in a notebook to display information about which versions of dependency package that was used to run the notebook.
Installation.

Run

    pip install git+https://github.com/jrjohansson/version_information
    
    
to install this extension

## Installation

In [1]:
%load_ext version_information

In [2]:
%version_information

Software versions
Python 3.5.1 64bit [GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
IPython 6.1.0
OS Darwin 15.6.0 x86_64 i386 64bit
Mon Sep 04 16:13:15 2017 CEST

# Native CPP codes

## Fibonacci CPP

In [3]:
!mkdir -p src

In [4]:
%%file src/fib.cpp
#define PY_ARRAY_UNIQUE_SYMBOL fkt_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>


struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};

inline npy_int64 fib_J(
      npy_int64 cn
);
inline npy_int64 fib_J(
      npy_int64 cn
) {
    if (cn < 2) {
        return cn;
    }
    return fib_J(cn - 1) + fib_J(cn - 2);    
}

#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
#include <signal.h>

void sighandler(int sig);

void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }


extern "C" {

    PyObject * create_signature;

    struct sigaction slot;

    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }

    PyObject * run(PyObject * self, PyObject * args) {
        {
            PyObject * pn; npy_int64 cn;
            if (
                PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 1
                and (pn = PyTuple_GET_ITEM(args, 0)) and PyLong_CheckExact(pn)
            ) {
                cn = PyLong_AS_LONG(pn);
                try {
                    return Py_BuildValue("l", fib_J(
                          cn
                    ));
                } catch (...) {
                    return NULL;
                }
            } else
                PyErr_Clear();
        }
        PyObject * signatures = Py_BuildValue("(sO)", "gANdcQBdcQFjaG9wZS5fYXN0ClZhcmlhYmxlCnECKYFxA31xBChYBAAAAGRpbXNxBUsAWAUAAABk\ndHlwZXEGY2J1aWx0aW5zCmludApxB1gEAAAAbmFtZXEIWAEAAABucQl1YmFhLg==\n", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for fib");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }

    PyMethodDef fibMethods[] = {
        { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
        { "run", run, METH_VARARGS, "module function" },
        { NULL, NULL, 0, NULL }
    };


    static struct PyModuleDef fibmodule = {
        PyModuleDef_HEAD_INIT,
        "fib",
        NULL,
        -1,
        fibMethods
    };



   PyMODINIT_FUNC PyInit_fib(void) {
        import_array();
        PyImport_ImportModule("numpy");
        return PyModule_Create(&fibmodule);
   }
}

Overwriting src/fib.cpp


## Quicksort CPP

In [5]:
%%file src/qsort_kernel.cpp
#define PY_ARRAY_UNIQUE_SYMBOL qsort_kernel_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
 typedef PyObject * ptr_t;
 typedef PyArrayObject * arrptr_t;
 PyObj(): dec(false), ptr(NULL) {}
 PyObj(ptr_t p): dec(false), ptr(p) {}
 ~PyObj() { if(dec) Py_DECREF(ptr); }
 PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
 PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
 operator bool() const { return ptr; }
 operator ptr_t() const { return ptr; }
 operator arrptr_t() const { return (arrptr_t)ptr; }
 bool dec;
 ptr_t ptr;
};
inline std::tuple<PyObject *, npy_intp const *, npy_double *> qsort_kernel_d1JJ(PyObject * pa,
    npy_intp const * __restrict__ sa,
                                         npy_double * __restrict__ ca,
                                         npy_int64 clo,
                                         npy_int64 chi);

inline std::tuple<PyObject *, npy_intp const *, npy_double *> qsort_kernel_d1JJ(PyObject * pa,
                                         npy_intp const * __restrict__ sa,
                                         npy_double * __restrict__ ca,
                                         npy_int64 clo,
                                         npy_int64 chi){
 npy_int64 ci = clo;
 npy_int64 cj = chi;
 npy_double cpivot;

 while (ci < chi) {
  cpivot = ca[(int)((clo + chi) / 2)];

  while (ci <= cj) {
   while (ca[ci] < cpivot) {
    ci += 1;
   }

   while (ca[cj] > cpivot) {
    cj -= 1;
   }

   if (ci <= cj) {
    auto ctmp = ca[ci];
    ca[ci] = ca[cj];
    ca[cj] = ctmp;
    ci += 1;
    cj -= 1;
   }
  }

  if (clo < cj) {
   qsort_kernel_d1JJ(pa, sa, ca, clo, cj);
  }

  clo = ci;
  cj = chi;
 }

 return std::make_tuple((PyObject *)pa, sa, ca);
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
 PyObject * create_signature;
 struct sigaction slot;
 PyObject * set_create_signature(PyObject * self, PyObject * args) {
  if (!PyArg_ParseTuple(args, "O", &create_signature)) {
   PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
   return NULL;
  }
  Py_INCREF(create_signature);
  memset(&slot, 0, sizeof(slot));
  slot.sa_handler = &sighandler;
  sigaction(SIGSEGV, &slot, NULL);
  sigaction(SIGBUS, &slot, NULL);
  Py_INCREF(Py_None);
  return Py_None;
 }
 PyObject * run(PyObject * self, PyObject * args) {
  {
   PyObj pa;
   PyObject * plo; npy_int64 clo;
   PyObject * phi; npy_int64 chi;
   if (
    PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 3
    and (pa = PyTuple_GET_ITEM(args, 0)) and PyArray_CheckExact(pa)
    and PyArray_TYPE((PyArrayObject *)pa) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pa) == 1
    and (plo = PyTuple_GET_ITEM(args, 1)) and PyLong_CheckExact(plo)
    and (phi = PyTuple_GET_ITEM(args, 2)) and PyLong_CheckExact(phi)
   ) {
    if (!(pa.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pa)))) {
     PyErr_SetString(PyExc_ValueError, "Invalid Argument type on a!");
     return NULL;
    }
    clo = PyLong_AS_LONG(plo);
    chi = PyLong_AS_LONG(phi);
    try {
     PyObject * res = std::get<0>(qsort_kernel_d1JJ(
        pa, PyArray_SHAPE((PyArrayObject *)pa), (npy_double *)PyArray_DATA((PyArrayObject *)pa)
      , clo
      , chi
     ));

     Py_INCREF(res);
     return res;
    } catch (...) {
     return NULL;
    }
   } else
    PyErr_Clear();
  }
  PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\nccopy_reg\n_reconstructor\np2\n(chope._ast\nVariable\np3\nc__builtin__\nobject\np4\nNtp5\nRp6\n(dp7\nS'name'\np8\nS'a'\np9\nsS'dtype'\np10\nS'float64'\np11\nsS'dims'\np12\nI1\nsbag2\n(g3\ng4\nNtp13\nRp14\n(dp15\ng8\nS'lo'\np16\nsg10\nc__builtin__\nint\np17\nsg12\nI0\nsbag2\n(g3\ng4\nNtp18\nRp19\n(dp20\ng8\nS'hi'\np21\nsg10\ng17\nsg12\nI0\nsbaa.", args);
  if (!signatures) {
   PyErr_SetString(PyExc_ValueError, "Error building signature string for qsort_kernel");
   return NULL;
  }
  return PyObject_Call(create_signature, signatures, NULL);
 }
 PyMethodDef qsort_kernelMethods[] = {
  { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
  { "run", run, METH_VARARGS, "module function" },
  { NULL, NULL, 0, NULL }
 };
}

static struct PyModuleDef qsort_kernel_module = {
    PyModuleDef_HEAD_INIT,
    "qsort_kernel",
    NULL,
    -1,
    qsort_kernelMethods
};

PyMODINIT_FUNC PyInit_qsort_kernel(void) {
    import_array();
    PyImport_ImportModule("numpy");
    return PyModule_Create(&qsort_kernel_module);
}

#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
 std::ostringstream buffer;
 buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
 void * stack[64];
 std::size_t depth = backtrace(stack, 64);
 if (!depth)
  buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
 else {
  char ** symbols = backtrace_symbols(stack, depth);
  for (std::size_t i = 1; i < depth; ++i) {
   std::string symbol = symbols[i];
    if (symbol.find_first_of(' ', 59) != std::string::npos) {
     std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
     int status;
     char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
     if (!status) {
      buffer << "    "
       << symbol.substr(0, 59)
       << demangled
       << symbol.substr(59 + name.size())
       << std::endl;
      free(demangled);
     } else
      buffer << "    " << symbol << std::endl;
    } else
     buffer << "    " << symbol << std::endl;
   }
   free(symbols);
  }
  std::cerr << buffer.str();
  std::exit(EXIT_FAILURE);
 }


Overwriting src/qsort_kernel.cpp


## Pi sum CPP

In [6]:
%%file src/pisum.cpp
#define PY_ARRAY_UNIQUE_SYMBOL pisum_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};

inline npy_double pisum_();

inline npy_double pisum_() {
    double csum = 0;
    int ck = 1;
    for (int cj = 1; cj < 501; ++cj) {
        csum = 0.0;
        for (ck = 1; ck < 10001; ++ck) {
            csum += (1.0 / (double)(ck * ck));
        }
    }

    return npy_double(csum);
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
    PyObject * create_signature;
    struct sigaction slot;
    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }
    PyObject * run(PyObject * self, PyObject * args) {
        {                try {
                    return Py_BuildValue("d", pisum_());
                } catch (...) {
                    return NULL;
                }
        }
        PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\na.", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for pisum");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }

PyMethodDef pisumMethods[] = {
{ "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
{ "run", run, METH_VARARGS, "module function" },
{ NULL, NULL, 0, NULL }
};

static struct PyModuleDef pisum_module = {
    PyModuleDef_HEAD_INIT,
    "pisum",
    NULL,
    -1,
    pisumMethods
};

PyMODINIT_FUNC PyInit_pisum(void) {
    import_array();
    PyImport_ImportModule("numpy");
    return PyModule_Create(&pisum_module);
}
}

#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }


Overwriting src/pisum.cpp


In [7]:
%%file src/pisum_opt.cpp
#define PY_ARRAY_UNIQUE_SYMBOL pisum_opt_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};

inline npy_double pisum_opt_();

inline npy_double pisum_opt_() {
    npy_double csum = npy_double();
    npy_intp ck = 1;
    npy_double cf = 0.0;

    for (npy_intp cj = 1; cj < 501; ++cj) {
        csum = 0.0;
        cf = 0.0;
        for (ck = 1; ck < 10001; ++ck) {
            cf += 1.0;
            auto c__sp0 = (cf * cf);
            csum += (1.0 / c__sp0);
        }
    }
    return csum;
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
    PyObject * create_signature;
    struct sigaction slot;
    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }
    PyObject * run(PyObject * self, PyObject * args) {
        {                try {
                    return Py_BuildValue("d", pisum_opt_());
                } catch (...) {
                    return NULL;
                }
        }
        PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\na.", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for pisum_opt");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }

    PyMethodDef pisum_optMethods[] = {
    { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
    { "run", run, METH_VARARGS, "module function" },
    { NULL, NULL, 0, NULL }
    };

    static struct PyModuleDef pisum_opt_module = {
        PyModuleDef_HEAD_INIT,
        "pisum_opt",
        NULL,
        -1,
        pisum_optMethods
    };

    PyMODINIT_FUNC PyInit_pisum_opt(void) {
        import_array();
        PyImport_ImportModule("numpy");
        return PyModule_Create(&pisum_opt_module);
    }
}

#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }


Overwriting src/pisum_opt.cpp


# 10th order poly log approx CPP

In [8]:
%%file src/ln.cpp
#define PY_ARRAY_UNIQUE_SYMBOL ln_hope_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};

inline void ln_hope_d1d1(PyObject * pX, npy_intp const * __restrict__ sX, npy_double * __restrict__ cX,
                                                    PyObject * pY, npy_intp const * __restrict__ sY, npy_double * __restrict__ cY);

inline void ln_hope_d1d1(PyObject * pX, npy_intp const * __restrict__ sX, npy_double * __restrict__ cX,
                                                    PyObject * pY, npy_intp const * __restrict__ sY, npy_double * __restrict__ cY){

    for (npy_intp i0 = 0; i0 < sY[0] - 0; ++i0) {
        cY[(int)(i0)] = (cX[i0] - 1) - (std::pow((cX[i0] - 1), 2) / 2) + (std::pow((cX[i0] - 1), 3) / 3) - (std::pow((cX[i0] - 1), 4) / 4) + (std::pow((cX[i0] - 1), 5) / 5) - (std::pow((cX[i0] - 1), 6) / 6) + (std::pow((cX[i0] - 1), 7) / 7) - (std::pow((cX[i0] - 1), 8) / 8) + (std::pow((cX[i0] - 1), 9) / 9);
    }
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
    PyObject * create_signature;
    struct sigaction slot;
    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }
    PyObject * run(PyObject * self, PyObject * args) {
        {
            PyObj pX;
            PyObj pY;
            if (
                PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 2
                and (pX = PyTuple_GET_ITEM(args, 0)) and PyArray_CheckExact(pX)
                and PyArray_TYPE((PyArrayObject *)pX) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pX) == 1
                and (pY = PyTuple_GET_ITEM(args, 1)) and PyArray_CheckExact(pY)
                and PyArray_TYPE((PyArrayObject *)pY) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pY) == 1
            ) {
                if (!(pX.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pX)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on X!");
                    return NULL;
                }
                if (!(pY.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pY)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on Y!");
                    return NULL;
                }
                try {
                    ln_hope_d1d1(
                          pX, PyArray_SHAPE((PyArrayObject *)pX), (npy_double *)PyArray_DATA((PyArrayObject *)pX)
                        , pY, PyArray_SHAPE((PyArrayObject *)pY), (npy_double *)PyArray_DATA((PyArrayObject *)pY)
                    );
                    Py_INCREF(Py_None);
                    return Py_None;
                } catch (...) {
                    return NULL;
                }
            } else
                PyErr_Clear();
        }
        PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\nccopy_reg\n_reconstructor\np2\n(chope._ast\nVariable\np3\nc__builtin__\nobject\np4\nNtp5\nRp6\n(dp7\nS'name'\np8\nS'X'\np9\nsS'dtype'\np10\nS'float64'\np11\nsS'dims'\np12\nI1\nsbag2\n(g3\ng4\nNtp13\nRp14\n(dp15\ng8\nS'Y'\np16\nsg10\ng11\nsg12\nI1\nsbaa.", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for ln_hope");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }

    PyMethodDef lnMethods[] = {
        { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
        { "run", run, METH_VARARGS, "module function" },
        { NULL, NULL, 0, NULL }
    };


    static struct PyModuleDef lnmodule = {
        PyModuleDef_HEAD_INIT,
        "ln",
        NULL,
        -1,
        lnMethods
    };



   PyMODINIT_FUNC PyInit_ln(void) {
        import_array();
        PyImport_ImportModule("numpy");
        return PyModule_Create(&lnmodule);
   }
}
#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }


Overwriting src/ln.cpp


In [9]:
%%file src/ln_exp.cpp
#define PY_ARRAY_UNIQUE_SYMBOL ln_hope_exp_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};

inline void ln_hope_exp_d1d1(PyObject * pX, npy_intp const * __restrict__ sX, npy_double * __restrict__ cX,
                                                    PyObject * pY, npy_intp const * __restrict__ sY, npy_double * __restrict__ cY);

inline void ln_hope_exp_d1d1(PyObject * pX, npy_intp const * __restrict__ sX, npy_double * __restrict__ cX,
                                                    PyObject * pY, npy_intp const * __restrict__ sY, npy_double * __restrict__ cY){

    for (npy_intp i0 = 0; i0 < sX[0] - 0; ++i0) {
        auto cx = (cX[i0] - 1);
        auto cx2 = (cx * cx);
        auto cx4 = (cx2 * cx2);
        auto cx6 = (cx4 * cx2);
        auto cx8 = (cx4 * cx4);
        cY[i0] = cx - (cx2 / 2) + (cx * cx2 / 3) - (cx4 / 4) + (cx * cx4 / 5) - (cx6 / 6) + (cx6 * cx / 7) - (cx8 / 8) + (cx8 * cx / 9);
    }
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
    PyObject * create_signature;
    struct sigaction slot;
    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }
    PyObject * run(PyObject * self, PyObject * args) {
        {
            PyObj pX;
            PyObj pY;
            if (
                PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 2
                and (pX = PyTuple_GET_ITEM(args, 0)) and PyArray_CheckExact(pX)
                and PyArray_TYPE((PyArrayObject *)pX) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pX) == 1
                and (pY = PyTuple_GET_ITEM(args, 1)) and PyArray_CheckExact(pY)
                and PyArray_TYPE((PyArrayObject *)pY) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pY) == 1
            ) {
                if (!(pX.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pX)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on X!");
                    return NULL;
                }
                if (!(pY.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pY)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on Y!");
                    return NULL;
                }
                try {
                    ln_hope_exp_d1d1(
                          pX, PyArray_SHAPE((PyArrayObject *)pX), (npy_double *)PyArray_DATA((PyArrayObject *)pX)
                        , pY, PyArray_SHAPE((PyArrayObject *)pY), (npy_double *)PyArray_DATA((PyArrayObject *)pY)
                    );
                    Py_INCREF(Py_None);
                    return Py_None;
                } catch (...) {
                    return NULL;
                }
            } else
                PyErr_Clear();
        }
        PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\nccopy_reg\n_reconstructor\np2\n(chope._ast\nVariable\np3\nc__builtin__\nobject\np4\nNtp5\nRp6\n(dp7\nS'name'\np8\nS'X'\np9\nsS'dtype'\np10\nS'float64'\np11\nsS'dims'\np12\nI1\nsbag2\n(g3\ng4\nNtp13\nRp14\n(dp15\ng8\nS'Y'\np16\nsg10\ng11\nsg12\nI1\nsbaa.", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for ln_hope_exp");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }
    PyMethodDef ln_expMethods[] = {
        { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
        { "run", run, METH_VARARGS, "module function" },
        { NULL, NULL, 0, NULL }
    };


    static struct PyModuleDef ln_expmodule = {
        PyModuleDef_HEAD_INIT,
        "ln_exp",
        NULL,
        -1,
        ln_expMethods
    };



   PyMODINIT_FUNC PyInit_ln_exp(void) {
        import_array();
        PyImport_ImportModule("numpy");
        return PyModule_Create(&ln_expmodule);
   }
}
#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }


Overwriting src/ln_exp.cpp


In [10]:
%%file src/ln_opt.cpp
#define PY_ARRAY_UNIQUE_SYMBOL ln_hope_opt_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};

inline void ln_hope_opt_d1d1(PyObject * pX, npy_intp const * __restrict__ sX, npy_double * __restrict__ cX,
                                                    PyObject * pY, npy_intp const * __restrict__ sY, npy_double * __restrict__ cY);

inline void ln_hope_opt_d1d1(PyObject * pX, npy_intp const * __restrict__ sX, npy_double * __restrict__ cX,
                                                    PyObject * pY, npy_intp const * __restrict__ sY, npy_double * __restrict__ cY){

    for (npy_intp i0 = 0; i0 < sY[0] - 0; ++i0) {
        auto c__sp0 = (cX[i0] * cX[i0]);
        auto c__sp1 = (c__sp0 * c__sp0);
        auto c__sp2 = (c__sp1 * c__sp1);
        auto c__sp3 = (c__sp2 * cX[i0]);
        auto c__sp4 = (c__sp0 * cX[i0]);
        auto c__sp5 = (c__sp4 * c__sp4);
        auto c__sp6 = (c__sp5 * cX[i0]);
        auto c__sp7 = (c__sp1 * cX[i0]);
        cY[(int)(i0)] = (-7129.0 / 2520.0) + (28 * c__sp4) + (-(18 * c__sp0)) + (-(9 * c__sp2 / 8)) + (-(14 * c__sp5)) + (-(63 * c__sp1 / 2)) + (126 * c__sp7 / 5) + (9 * cX[i0]) + (c__sp3 / 9) + (36 * c__sp6 / 7);
    }
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
    PyObject * create_signature;
    struct sigaction slot;
    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }
    PyObject * run(PyObject * self, PyObject * args) {
        {
            PyObj pX;
            PyObj pY;
            if (
                PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 2
                and (pX = PyTuple_GET_ITEM(args, 0)) and PyArray_CheckExact(pX)
                and PyArray_TYPE((PyArrayObject *)pX) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pX) == 1
                and (pY = PyTuple_GET_ITEM(args, 1)) and PyArray_CheckExact(pY)
                and PyArray_TYPE((PyArrayObject *)pY) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pY) == 1
            ) {
                if (!(pX.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pX)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on X!");
                    return NULL;
                }
                if (!(pY.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pY)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on Y!");
                    return NULL;
                }
                try {
                    ln_hope_opt_d1d1(
                          pX, PyArray_SHAPE((PyArrayObject *)pX), (npy_double *)PyArray_DATA((PyArrayObject *)pX)
                        , pY, PyArray_SHAPE((PyArrayObject *)pY), (npy_double *)PyArray_DATA((PyArrayObject *)pY)
                    );
                    Py_INCREF(Py_None);
                    return Py_None;
                } catch (...) {
                    return NULL;
                }
            } else
                PyErr_Clear();
        }
        PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\nccopy_reg\n_reconstructor\np2\n(chope._ast\nVariable\np3\nc__builtin__\nobject\np4\nNtp5\nRp6\n(dp7\nS'name'\np8\nS'X'\np9\nsS'dtype'\np10\nS'float64'\np11\nsS'dims'\np12\nI1\nsbag2\n(g3\ng4\nNtp13\nRp14\n(dp15\ng8\nS'Y'\np16\nsg10\ng11\nsg12\nI1\nsbaa.", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for ln_hope_opt");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }

    PyMethodDef ln_optMethods[] = {
        { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
        { "run", run, METH_VARARGS, "module function" },
        { NULL, NULL, 0, NULL }
    };


    static struct PyModuleDef ln_optmodule = {
        PyModuleDef_HEAD_INIT,
        "ln",
        NULL,
        -1,
        ln_optMethods
    };



   PyMODINIT_FUNC PyInit_ln_opt(void) {
        import_array();
        PyImport_ImportModule("numpy");
        return PyModule_Create(&ln_optmodule);
   }
}
#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }


Overwriting src/ln_opt.cpp


## Simplify CPP

In [11]:
%%file src/poly.cpp
#define PY_ARRAY_UNIQUE_SYMBOL poly_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};
inline void poly_d1d1(PyObject * pres, npy_intp const * __restrict__ sres, npy_double * __restrict__ cres,
                                            PyObject * parg, npy_intp const * __restrict__ sarg, npy_double * __restrict__ carg);

inline void poly_d1d1(PyObject * pres, npy_intp const * __restrict__ sres, npy_double * __restrict__ cres,
                                            PyObject * parg, npy_intp const * __restrict__ sarg, npy_double * __restrict__ carg){

    npy_double arg_i;
    double sin_arg_i;
    for (npy_intp i0 = 0; i0 < sres[0] - 0; ++i0) {
        arg_i = carg[i0];
        cres[(int)(i0)] = std::pow(std::sin(arg_i), 2) + (std::pow(arg_i, 3) + std::pow(arg_i, 2) - arg_i - 1) / (std::pow(arg_i, 2) + 2 * arg_i + 1) + std::pow(std::cos(arg_i), 2);
    }
}
void sighandler(int sig);
#include <signal.h>
extern "C" {
    PyObject * create_signature;
    struct sigaction slot;
    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }
    PyObject * run(PyObject * self, PyObject * args) {
        {
            PyObj pres;
            PyObj parg;
            if (
                PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 2
                and (pres = PyTuple_GET_ITEM(args, 0)) and PyArray_CheckExact(pres)
                and PyArray_TYPE((PyArrayObject *)pres) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pres) == 1
                and (parg = PyTuple_GET_ITEM(args, 1)) and PyArray_CheckExact(parg)
                and PyArray_TYPE((PyArrayObject *)parg) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)parg) == 1
            ) {
                if (!(pres.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pres)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on res!");
                    return NULL;
                }
                if (!(parg.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)parg)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on arg!");
                    return NULL;
                }
                try {
                    poly_d1d1(
                          pres, PyArray_SHAPE((PyArrayObject *)pres), (npy_double *)PyArray_DATA((PyArrayObject *)pres)
                        , parg, PyArray_SHAPE((PyArrayObject *)parg), (npy_double *)PyArray_DATA((PyArrayObject *)parg)
                    );
                    Py_INCREF(Py_None);
                    return Py_None;
                } catch (...) {
                    return NULL;
                }
            } else
                PyErr_Clear();
        }
        PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\nccopy_reg\n_reconstructor\np2\n(chope._ast\nVariable\np3\nc__builtin__\nobject\np4\nNtp5\nRp6\n(dp7\nS'name'\np8\nS'res'\np9\nsS'dtype'\np10\nS'float64'\np11\nsS'dims'\np12\nI1\nsbag2\n(g3\ng4\nNtp13\nRp14\n(dp15\ng8\nS'arg'\np16\nsg10\ng11\nsg12\nI1\nsbaa.", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for poly");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }

    PyMethodDef polyMethods[] = {
        { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
        { "run", run, METH_VARARGS, "module function" },
        { NULL, NULL, 0, NULL }
    };

    static struct PyModuleDef polymodule = {
        PyModuleDef_HEAD_INIT,
        "poly",
        NULL,
        -1,
        polyMethods
    };

   PyMODINIT_FUNC PyInit_poly(void) {
        import_array();
        PyImport_ImportModule("numpy");
        return PyModule_Create(&polymodule);
   }
}
#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }


Overwriting src/poly.cpp


In [12]:
%%file src/poly_opt.cpp
#define PY_ARRAY_UNIQUE_SYMBOL poly_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
	typedef PyObject * ptr_t;
	typedef PyArrayObject * arrptr_t;
	PyObj(): dec(false), ptr(NULL) {}
	PyObj(ptr_t p): dec(false), ptr(p) {}
	~PyObj() { if(dec) Py_DECREF(ptr); }
	PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
	PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
	operator bool() const { return ptr; }
	operator ptr_t() const { return ptr; }
	operator arrptr_t() const { return (arrptr_t)ptr; }
	bool dec;
	ptr_t ptr;
};

inline void poly_d1d1(PyObject * pres, npy_intp const * __restrict__ sres, npy_double * __restrict__ cres, 
											PyObject * parg, npy_intp const * __restrict__ sarg, npy_double * __restrict__ carg);

inline void poly_d1d1(PyObject * pres, npy_intp const * __restrict__ sres, npy_double * __restrict__ cres, 
											PyObject * parg, npy_intp const * __restrict__ sarg, npy_double * __restrict__ carg){
	for (npy_intp i0 = 0; i0 < sres[0] - 0; ++i0) {
		cres[(int)(i0)] = carg[i0];
	}
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
	PyObject * create_signature;
	struct sigaction slot;
	PyObject * set_create_signature(PyObject * self, PyObject * args) {
		if (!PyArg_ParseTuple(args, "O", &create_signature)) {
			PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
			return NULL;
		}
		Py_INCREF(create_signature);
		memset(&slot, 0, sizeof(slot));
		slot.sa_handler = &sighandler;
		sigaction(SIGSEGV, &slot, NULL);
		sigaction(SIGBUS, &slot, NULL);
		Py_INCREF(Py_None);
		return Py_None;
	}
	PyObject * run(PyObject * self, PyObject * args) {
		{
			PyObj pres;
			PyObj parg;
			if (
				PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 2
				and (pres = PyTuple_GET_ITEM(args, 0)) and PyArray_CheckExact(pres)
				and PyArray_TYPE((PyArrayObject *)pres) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pres) == 1
				and (parg = PyTuple_GET_ITEM(args, 1)) and PyArray_CheckExact(parg)
				and PyArray_TYPE((PyArrayObject *)parg) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)parg) == 1
			) {
				if (!(pres.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pres)))) {
					PyErr_SetString(PyExc_ValueError, "Invalid Argument type on res!");
					return NULL;
				}
				if (!(parg.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)parg)))) {
					PyErr_SetString(PyExc_ValueError, "Invalid Argument type on arg!");
					return NULL;
				}
				try {
					poly_d1d1(
						  pres, PyArray_SHAPE((PyArrayObject *)pres), (npy_double *)PyArray_DATA((PyArrayObject *)pres)
						, parg, PyArray_SHAPE((PyArrayObject *)parg), (npy_double *)PyArray_DATA((PyArrayObject *)parg)
					);
					Py_INCREF(Py_None);
					return Py_None;
				} catch (...) {
					return NULL;
				}
			} else
				PyErr_Clear();
		}
		PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\nccopy_reg\n_reconstructor\np2\n(chope._ast\nVariable\np3\nc__builtin__\nobject\np4\nNtp5\nRp6\n(dp7\nS'name'\np8\nS'res'\np9\nsS'dtype'\np10\nS'float64'\np11\nsS'dims'\np12\nI1\nsbag2\n(g3\ng4\nNtp13\nRp14\n(dp15\ng8\nS'arg'\np16\nsg10\ng11\nsg12\nI1\nsbaa.", args);
		if (!signatures) {
			PyErr_SetString(PyExc_ValueError, "Error building signature string for poly");
			return NULL;
		}
		return PyObject_Call(create_signature, signatures, NULL);
	}
	PyMethodDef polyMethods[] = {
		{ "set_create_signature", (PyCFunction)set_create_signature, METH_VARARGS },
		{ "run", (PyCFunction)run, METH_VARARGS },
		{ NULL, NULL }
	};
	PyMODINIT_FUNC initpoly(void) {
		import_array();
		PyImport_ImportModule("numpy");
		(void)Py_InitModule("poly", polyMethods);
	}
}
#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
	std::ostringstream buffer;
	buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
	void * stack[64];
	std::size_t depth = backtrace(stack, 64);
	if (!depth)
		buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
	else {
		char ** symbols = backtrace_symbols(stack, depth);
		for (std::size_t i = 1; i < depth; ++i) {
			std::string symbol = symbols[i];
				if (symbol.find_first_of(' ', 59) != std::string::npos) {
					std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
					int status;
					char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
					if (!status) {
						buffer << "    " 
							<< symbol.substr(0, 59) 
							<< demangled
							<< symbol.substr(59 + name.size())
							<< std::endl;
						free(demangled);
					} else
						buffer << "    " << symbol << std::endl;
				} else
					buffer << "    " << symbol << std::endl;
			}
			free(symbols);
		}
		std::cerr << buffer.str();
		std::exit(EXIT_FAILURE);
	}


Overwriting src/poly_opt.cpp


## Pairwise distance

In [13]:
%%file src/pairwise.cpp
#define PY_ARRAY_UNIQUE_SYMBOL pairwise_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};

inline void pairwise_d2d2JJ(PyObject * pX,
                                                        npy_intp const * __restrict__ sX,
                                                        npy_double * __restrict__ cX,
                                                        PyObject * pD,
                                                        npy_intp const * __restrict__ sD,
                                                        npy_double * __restrict__ cD,
                                                        npy_int64 const cM,
                                                        npy_int64 const cN);

inline void pairwise_d2d2JJ(PyObject * pX,
                                                        npy_intp const * __restrict__ sX,
                                                        npy_double * __restrict__ cX,
                                                        PyObject * pD,
                                                        npy_intp const * __restrict__ sD,
                                                        npy_double * __restrict__ cD,
                                                        npy_int64 const cM,
                                                        npy_int64 const cN){

    npy_double cd = 0.0;
    npy_double ctmp = 0.0;

    npy_intp cj;
    npy_intp ck;
    npy_intp x_i_idx;
    npy_intp x_j_idx;
    npy_intp d_i_idx;
    npy_intp const xp = sX[1];
    npy_intp const dp = sD[1];

    for (npy_intp ci = 0; ci < cM; ++ci) {
        x_i_idx = ci*xp;
        d_i_idx = ci*dp;
        for (cj = 0; cj < cM; ++cj) {
            cd = 0.0;
            x_j_idx = cj*xp;
            for (ck = 0; ck < cN; ++ck) {
                ctmp = (cX[(x_i_idx + ck)] - cX[(x_j_idx + ck)]);
                cd += (ctmp * ctmp);
            }
            cD[(d_i_idx + cj)] = std::sqrt(cd);
        }
    }
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
    PyObject * create_signature;
    struct sigaction slot;
    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }
    PyObject * run(PyObject * self, PyObject * args) {
        {
            PyObj pX;
            PyObj pD;
            PyObject * pM; npy_int64 cM;
            PyObject * pN; npy_int64 cN;
            if (
                PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 4
                and (pX = PyTuple_GET_ITEM(args, 0)) and PyArray_CheckExact(pX)
                and PyArray_TYPE((PyArrayObject *)pX) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pX) == 2
                and (pD = PyTuple_GET_ITEM(args, 1)) and PyArray_CheckExact(pD)
                and PyArray_TYPE((PyArrayObject *)pD) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pD) == 2
                and (pM = PyTuple_GET_ITEM(args, 2)) and PyLong_CheckExact(pM)
                and (pN = PyTuple_GET_ITEM(args, 3)) and PyLong_CheckExact(pN)
            ) {
                if (!(pX.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pX)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on X!");
                    return NULL;
                }
                if (!(pD.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pD)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on D!");
                    return NULL;
                }
                cM = PyLong_AS_LONG(pM);
                cN = PyLong_AS_LONG(pN);
                try {
                    pairwise_d2d2JJ(
                          pX, PyArray_SHAPE((PyArrayObject *)pX), (npy_double *)PyArray_DATA((PyArrayObject *)pX)
                        , pD, PyArray_SHAPE((PyArrayObject *)pD), (npy_double *)PyArray_DATA((PyArrayObject *)pD)
                        , cM
                        , cN
                    );
                    Py_INCREF(Py_None);
                    return Py_None;
                } catch (...) {
                    return NULL;
                }
            } else
                PyErr_Clear();
        }
        PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\nccopy_reg\n_reconstructor\np2\n(chope._ast\nVariable\np3\nc__builtin__\nobject\np4\nNtp5\nRp6\n(dp7\nS'name'\np8\nS'X'\np9\nsS'dtype'\np10\nS'float64'\np11\nsS'dims'\np12\nI2\nsbag2\n(g3\ng4\nNtp13\nRp14\n(dp15\ng8\nS'D'\np16\nsg10\ng11\nsg12\nI2\nsbag2\n(g3\ng4\nNtp17\nRp18\n(dp19\ng8\nS'M'\np20\nsg10\nc__builtin__\nint\np21\nsg12\nI0\nsbag2\n(g3\ng4\nNtp22\nRp23\n(dp24\ng8\nS'N'\np25\nsg10\ng21\nsg12\nI0\nsbaa.", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for pairwise");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }

    PyMethodDef pairwiseMethods[] = {
        { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
        { "run", run, METH_VARARGS, "module function" },
        { NULL, NULL, 0, NULL }
    };


    static struct PyModuleDef pairwisemodule = {
        PyModuleDef_HEAD_INIT,
        "pairwise",
        NULL,
        -1,
        pairwiseMethods
    };


   PyMODINIT_FUNC PyInit_pairwise(void) {
        import_array();
        PyImport_ImportModule("numpy");
        return PyModule_Create(&pairwisemodule);
   }
}
#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }


Overwriting src/pairwise.cpp


Star point spread function CPP

In [14]:
%%file src/pdf.cpp
#define PY_ARRAY_UNIQUE_SYMBOL pdf_ARRAY_API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>
#include <numpy/arrayscalars.h>
#include <cmath>
#include <tuple>
#include <numeric>
#include <cstdint>
#include <cstdlib>
#include <exception>
#include <functional>
#include <type_traits>
struct PyObj {
    typedef PyObject * ptr_t;
    typedef PyArrayObject * arrptr_t;
    PyObj(): dec(false), ptr(NULL) {}
    PyObj(ptr_t p): dec(false), ptr(p) {}
    ~PyObj() { if(dec) Py_DECREF(ptr); }
    PyObj & operator=(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = false; return *this; }
    PyObj & incref(ptr_t p) { if(dec) Py_DECREF(ptr); ptr = p; dec = (p != NULL); return *this; }
    operator bool() const { return ptr; }
    operator ptr_t() const { return ptr; }
    operator arrptr_t() const { return (arrptr_t)ptr; }
    bool dec;
    ptr_t ptr;
};

inline std::tuple<PyObject *, npy_intp const *, npy_float *> pdf_f2l1d1f2JDd(
      PyObject * pdensity
  , npy_intp const * __restrict__ sdensity
  , npy_float * __restrict__ cdensity
    , PyObject * pdims
    , npy_intp const * __restrict__ sdims
    , npy_int64 * __restrict__ cdims
    , PyObject * pcenter
    , npy_intp const * __restrict__ scenter
    , npy_double * __restrict__ ccenter
    , PyObject * pw2D
    , npy_intp const * __restrict__ sw2D
    , npy_float * __restrict__ cw2D
    , npy_int64 cr50
    , npy_double cb
    , npy_double ca);

inline std::tuple<PyObject *, npy_intp const *, npy_float *> pdf_f2l1d1f2JDd(
      PyObject * pdensity
  , npy_intp const * __restrict__ sdensity
  , npy_float * __restrict__ cdensity
    , PyObject * pdims
    , npy_intp const * __restrict__ sdims
    , npy_int64 * __restrict__ cdims
    , PyObject * pcenter
    , npy_intp const * __restrict__ scenter
    , npy_double * __restrict__ ccenter
    , PyObject * pw2D
    , npy_intp const * __restrict__ sw2D
    , npy_float * __restrict__ cw2D
    , npy_int64 const cr50
    , npy_double const cb
    , npy_double const ca) {

    npy_double cdr;
    npy_double c__sum0;
    const npy_double x_center = ccenter[0];
    const npy_double y_center = ccenter[1];
    const npy_intp len_0_sw2D = sw2D[0];
    const npy_intp len_1_sw2D = sw2D[1];
    npy_intp dc[] = {len_0_sw2D, len_1_sw2D};
    PyObject * pc = PyArray_EMPTY(2, dc, NPY_FLOAT64, 0);
    npy_intp * sc = PyArray_SHAPE((PyArrayObject *)pc);
    npy_double * cc = (npy_double *)PyArray_DATA((PyArrayObject *)pc);

    npy_intp cy = 0;
    npy_intp i0 = 0;
    npy_intp i1 = 0;
    npy_intp i2 = 0;
    npy_intp i3 = 0;

    npy_intp sw2D_i0_idx;
    npy_intp sw2D_i2_idx;
    npy_intp density_x_idx;

    auto c__sp0 = ca * ca;
    auto c__sp1 = cr50 * cr50;
    auto c__sp2 = 1.0 / (c__sp0 * c__sp1);
    auto c__sp4 = 0.3183098861846737 * c__sp2 * (-1 + cb);

    for (npy_intp cx = 0; cx < cdims[(int)(0)]; ++cx) {

        density_x_idx = cx*sdensity[1];
        for (cy = 0; cy < cdims[(int)(1)]; ++cy) {

            cdr = std::sqrt(std::pow(cx - x_center, 2) + std::pow(cy - y_center, 2));
            auto c__sp3 = cdr * cdr;
            auto c__sp5 = c__sp4 * std::pow((1 + c__sp2 * c__sp3), -cb);

            for (i0 = 0; i0 < len_0_sw2D - 0; ++i0) {

                sw2D_i0_idx = (i0)*len_1_sw2D;
                for (i1 = 0; i1 < len_1_sw2D - 0; ++i1) {
                    cc[sw2D_i0_idx + i1] = c__sp5 * cw2D[sw2D_i0_idx + i1];
                }
            }

            c__sum0 = 0;
            for (i2 = 0; i2 < len_0_sw2D - 0; ++i2) {

                sw2D_i2_idx = (i2)*len_1_sw2D;
                for (i3 = 0; i3 < len_1_sw2D - 0; ++i3) {
                    c__sum0 += cc[sw2D_i2_idx + i3];
                }
            }

            cdensity[(density_x_idx + cy)] = c__sum0;
        }
    }
    return std::make_tuple((PyObject *)pdensity, sdensity, cdensity);
}

void sighandler(int sig);
#include <signal.h>
extern "C" {
    PyObject * create_signature;
    struct sigaction slot;
    PyObject * set_create_signature(PyObject * self, PyObject * args) {
        if (!PyArg_ParseTuple(args, "O", &create_signature)) {
            PyErr_SetString(PyExc_ValueError, "Invalid Argument to set_create_signature!");
            return NULL;
        }
        Py_INCREF(create_signature);
        memset(&slot, 0, sizeof(slot));
        slot.sa_handler = &sighandler;
        sigaction(SIGSEGV, &slot, NULL);
        sigaction(SIGBUS, &slot, NULL);
        Py_INCREF(Py_None);
        return Py_None;
    }
    PyObject * run(PyObject * self, PyObject * args) {
        {
            PyObj pdensity;
            PyObj pdims;
            PyObj pcenter;
            PyObj pw2D;
            PyObject * pr50; npy_int64 cr50;
            PyObject * pb; npy_double cb;
            PyObject * pa; npy_double ca;
            if (
                PyTuple_CheckExact(args) and PyTuple_GET_SIZE(args) == 7
                and (pdensity = PyTuple_GET_ITEM(args, 0)) and PyArray_CheckExact(pdensity)
                and PyArray_TYPE((PyArrayObject *)pdensity) == NPY_FLOAT32 and PyArray_NDIM((PyArrayObject *)pdensity) == 2
                and (pdims = PyTuple_GET_ITEM(args, 1)) and PyArray_CheckExact(pdims)
                and PyArray_TYPE((PyArrayObject *)pdims) == NPY_INT64 and PyArray_NDIM((PyArrayObject *)pdims) == 1
                and (pcenter = PyTuple_GET_ITEM(args, 2)) and PyArray_CheckExact(pcenter)
                and PyArray_TYPE((PyArrayObject *)pcenter) == NPY_FLOAT64 and PyArray_NDIM((PyArrayObject *)pcenter) == 1
                and (pw2D = PyTuple_GET_ITEM(args, 3)) and PyArray_CheckExact(pw2D)
                and PyArray_TYPE((PyArrayObject *)pw2D) == NPY_FLOAT32 and PyArray_NDIM((PyArrayObject *)pw2D) == 2
                and (pr50 = PyTuple_GET_ITEM(args, 4)) and PyLong_CheckExact(pr50)
                and (pb = PyTuple_GET_ITEM(args, 5)) and PyFloat_CheckExact(pb)
                and (pa = PyTuple_GET_ITEM(args, 6)) and PyArray_IsScalar(pa, Double)
            ) {
                if (!(pdensity.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pdensity)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on density!");
                    return NULL;
                }
                if (!(pdims.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pdims)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on dims!");
                    return NULL;
                }
                if (!(pcenter.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pcenter)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on center!");
                    return NULL;
                }
                if (!(pw2D.incref((PyObject *)PyArray_GETCONTIGUOUS((PyArrayObject *)pw2D)))) {
                    PyErr_SetString(PyExc_ValueError, "Invalid Argument type on w2D!");
                    return NULL;
                }
                cr50 = PyLong_AS_LONG(pr50);
                cb = PyFloat_AS_DOUBLE(pb);
                ca = PyArrayScalar_VAL(pa, Double);
                try {
                    PyObject * res = std::get<0>(pdf_f2l1d1f2JDd(
                          pdensity, PyArray_SHAPE((PyArrayObject *)pdensity), (npy_float *)PyArray_DATA((PyArrayObject *)pdensity)
                        , pdims, PyArray_SHAPE((PyArrayObject *)pdims), (npy_int64 *)PyArray_DATA((PyArrayObject *)pdims)
                        , pcenter, PyArray_SHAPE((PyArrayObject *)pcenter), (npy_double *)PyArray_DATA((PyArrayObject *)pcenter)
                        , pw2D, PyArray_SHAPE((PyArrayObject *)pw2D), (npy_float *)PyArray_DATA((PyArrayObject *)pw2D)
                        , cr50
                        , cb
                        , ca
                    ));

                    Py_INCREF(res);
                    return res;
                } catch (...) {
                    return NULL;
                }
            } else
                PyErr_Clear();
        }
        PyObject * signatures = Py_BuildValue("(sO)", "(lp0\n(lp1\nccopy_reg\n_reconstructor\np2\n(chope._ast\nVariable\np3\nc__builtin__\nobject\np4\nNtp5\nRp6\n(dp7\nS'name'\np8\nS'density'\np9\nsS'dtype'\np10\nS'float32'\np11\nsS'dims'\np12\nI2\nsbag2\n(g3\ng4\nNtp13\nRp14\n(dp15\ng8\ng12\nsg10\nS'int64'\np16\nsg12\nI1\nsbag2\n(g3\ng4\nNtp17\nRp18\n(dp19\ng8\nS'center'\np20\nsg10\nS'float64'\np21\nsg12\nI1\nsbag2\n(g3\ng4\nNtp22\nRp23\n(dp24\ng8\nS'w2D'\np25\nsg10\ng11\nsg12\nI2\nsbag2\n(g3\ng4\nNtp26\nRp27\n(dp28\ng8\nS'r50'\np29\nsg10\nc__builtin__\nint\np30\nsg12\nI0\nsbag2\n(g3\ng4\nNtp31\nRp32\n(dp33\ng8\nS'b'\np34\nsg10\nc__builtin__\nfloat\np35\nsg12\nI0\nsbag2\n(g3\ng4\nNtp36\nRp37\n(dp38\ng8\nS'a'\np39\nsg10\ng21\nsg12\nI0\nsbaa.", args);
        if (!signatures) {
            PyErr_SetString(PyExc_ValueError, "Error building signature string for pdf");
            return NULL;
        }
        return PyObject_Call(create_signature, signatures, NULL);
    }

    PyMethodDef pdfMethods[] = {
        { "set_create_signature", set_create_signature, METH_VARARGS, "signal handler" },
        { "run", run, METH_VARARGS, "module function" },
        { NULL, NULL, 0, NULL }
    };

    static struct PyModuleDef pdfmodule = {
        PyModuleDef_HEAD_INIT,
        "pdf",
        NULL,
        -1,
        pdfMethods
    };

   PyMODINIT_FUNC PyInit_pdf(void) {
        import_array();
        PyImport_ImportModule("numpy");
        return PyModule_Create(&pdfmodule);
   }
}
#include <string>
#include <sstream>
#include <iostream>
#include <cxxabi.h>
#include <execinfo.h>
void sighandler(int sig) {
    std::ostringstream buffer;
    buffer << "Abort by " << (sig == SIGSEGV ? "segfault" : "bus error") << std::endl;
    void * stack[64];
    std::size_t depth = backtrace(stack, 64);
    if (!depth)
        buffer << "  <empty stacktrace, possibly corrupt>" << std::endl;
    else {
        char ** symbols = backtrace_symbols(stack, depth);
        for (std::size_t i = 1; i < depth; ++i) {
            std::string symbol = symbols[i];
                if (symbol.find_first_of(' ', 59) != std::string::npos) {
                    std::string name = symbol.substr(59, symbol.find_first_of(' ', 59) - 59);
                    int status;
                    char * demangled = abi::__cxa_demangle(name.c_str(), NULL, NULL, &status);
                    if (!status) {
                        buffer << "    "
                            << symbol.substr(0, 59)
                            << demangled
                            << symbol.substr(59 + name.size())
                            << std::endl;
                        free(demangled);
                    } else
                        buffer << "    " << symbol << std::endl;
                } else
                    buffer << "    " << symbol << std::endl;
            }
            free(symbols);
        }
        std::cerr << buffer.str();
        std::exit(EXIT_FAILURE);
    }

Overwriting src/pdf.cpp


# Python helper module

In [15]:
%%file native_util.py
import importlib
import os
import glob
import sys

from numpy.distutils.misc_util import get_numpy_include_dirs
import setuptools
from os import listdir
# import tempfile
from hope import config

try:
    # python 3
    import io
    file = io.IOBase
except ImportError:
    pass


def load(name):
    compile(name, "./src")
    module = importlib.import_module(name)
    # module = __import__(name, globals(), locals(), [], -1)
    return module



def compile(name, src_folder, target_folder = "./"):
    localfilename =os.path.join(src_folder, name)
    
    outfile, stdout, stderr, argv = None, None, None, sys.argv
    try:
        sys.stdout.flush(), sys.stderr.flush()
        outpath = os.path.join(target_folder, "{0}.out".format(localfilename))
        if os.path.exists(outpath):
            os.remove(outpath)
            
        so_path = os.path.join(target_folder, "{0}.so".format(name))
        if os.path.exists(so_path):
            os.remove(so_path)
            
        outfile = open(outpath, 'w')
        
        try:
            sys.stdout.fileno()
            sys.stderr.fileno()
            have_fileno = True
        except OSError:
            have_fileno = False
        
        if have_fileno and isinstance(sys.stdout, file) and  isinstance(sys.stdout, file):
            stdout, stderr = os.dup(sys.stdout.fileno()), os.dup(sys.stderr.fileno())
            os.dup2(outfile.fileno(), sys.stdout.fileno())
            os.dup2(outfile.fileno(), sys.stderr.fileno())
        else:
            stdout, stderr = sys.stdout, sys.stderr
            sys.stdout, sys.stderr = outfile, outfile
        try:
            sources = "./{0}.cpp".format(localfilename)
            sys.argv = ["", "build_ext",
                        "-b", target_folder,  #--build-lib (-b)     directory for compiled extension modules
                        "-t", "." #--build-temp - a rel path will result in a dir structure of -b at the cur position 
                        ]
    
            localfilename = str(localfilename)
            sources = str(sources)
    
            setuptools.setup( \
                  name = name\
                , ext_modules = [setuptools.Extension( \
                      name \
                    , sources = [sources] \
                    , extra_compile_args = config.cxxflags \
                  )] \
                , include_dirs = get_numpy_include_dirs() \
            )
        except SystemExit as e:
            print(sys.stderr.write(str(e)))
        sys.stdout.flush(), sys.stderr.flush()
    finally:
        if isinstance(stdout, int):
            os.dup2(stdout, sys.stdout.fileno()), os.close(stdout)
        elif not stdout is None:
            sys.stdout = stdout
        if isinstance(stderr, int):
            os.dup2(stderr, sys.stderr.fileno()), os.close(stderr)
        elif not stderr is None:
            sys.stderr = stderr
        if isinstance(outfile, file):
            outfile.close()
        sys.argv = argv
    
    with open(outpath) as outfile:
        out = outfile.read()
    
    modules = glob.glob(os.path.join(target_folder, "{}*.so".format(name)))
        
    if not modules or out.find("error:") > -1:
        print(out)
        raise Exception("Error compiling function {0} (compiled to {1})".format(localfilename, target_folder))
    
    if out.find("warning:") > -1:
        import warnings
        warnings.warn("A warning has been issued during compilation:\n%s"%out)
    
    print(out)


def compile_all():
    src_folder = "./src"
    func_names = (src_file.split(".cpp")[0] for src_file in listdir(src_folder) if src_file.endswith(".cpp"))
    for func_name in func_names:
        compile(func_name, src_folder)
    
    
if __name__ == '__main__':
    compile_all()

Overwriting native_util.py


In [16]:
%%file util.py
# Copyright (C) 2014 ETH Zurich, Institute for Astronomy

'''
Created on Aug 4, 2014

author: jakeret
'''
from __future__ import print_function, division, absolute_import, unicode_literals
import math

def perf_comp_data(func_list, data_list, rep=5, number=1, extra_setup=None):
    ''' Function to compare the performance of different functions.
    
    Parameters
    ==========
    func_list : list
        list with function names as strings
    data_list : list
        list with data set names as strings
    rep : int
        number of repetitions of the whole comparison
    number : int
        number of executions for every function
    '''
    from timeit import repeat
    res_list = {}
    for name in enumerate(func_list):
        if data_list is None:
            stmt = "%s()"%(name[1])
            setup = "from __main__ import %s"%(name[1]) 
        else:
            stmt = "%s(%s)"%(name[1], data_list[name[0]])
            setup = "from __main__ import %s, %s"%(name[1], data_list[name[0]])
        if extra_setup is not None:
            stmt = extra_setup + "; " + stmt
              
        results = repeat(stmt=stmt, setup=setup, repeat=rep, number=number)
        
        res_list[name[1]] = (median(results), min(results))
        
#     res_sort = sorted(res_list.iteritems(), key=lambda (k, v): (v, k))
    res_sort = sorted(iter(res_list.items()), key=lambda k_v: (k_v[1], k_v[0]))
    for func, (av_time, min_time) in res_sort:
        rel = av_time / res_sort[0][1][0]
        print('function: {0!s:20}, av. time sec: {1:>12.8f}, min. time sec: {2:>12.8f}, relative: {3:>9.1f}'.format(func, av_time, min_time, rel))

def median(x):
    s_x = sorted(x)
    return (s_x[int(math.floor((len(s_x)-1)/2))] + s_x[int(math.ceil((len(s_x)-1)/2))])/2

Overwriting util.py
